In [9]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta
import sqlalchemy as sa
from configparser import ConfigParser

In [10]:
def get_data(base_url, endpoint, params= None):
  try:
    if params == None:
      endpoint_url = f"{base_url}/{endpoint}"
    else:
      endpoint_url = f"{base_url}/{endpoint}/{params['start']}/{params['end']}" # Condicional creada en el caso que un endpoint requiera parámetros de tiempo.

    response = requests.get(endpoint_url, params=params)
    response.raise_for_status()  # Levanta una excepción si hay un error en la respuesta HTTP.

    try: # Verificar si los datos están en formato JSON.
          data = response.json()
          data = data["data"]
    except:
        print("El formato de respuesta no es el esperado")
        return None
    return data
  
  except requests.exceptions.RequestException as e: # Capturar cualquier error de solicitud, como errores HTTP.
      print(f"La petición ha fallado. Código de error : {e}")
      return None

In [11]:
def build_table(json_data):
  df = pd.json_normalize(json_data)
  return df

In [12]:
def date_fromTo(number_days = None):
  if number_days == None:
    end_date = datetime.utcnow() - timedelta(days=1)
    start_date = datetime.utcnow() - timedelta(days=31) # Fecha de inicio: 31 dias antes que la ejecucion del script.
  else:
    end_date = datetime.utcnow() - timedelta(days=1)
    start_date = datetime.utcnow() - timedelta(number_days)

  end_date = end_date.strftime("%Y-%m-%dT%H:00:00Z")
  start_date = start_date.strftime("%Y-%m-%dT%H:00:00Z")

  params = {
    "start": start_date,
    "end": end_date
    }
  return params

In [13]:
def save_to_csv(df, save_path):
  
  """
  Funcion que almacena un DataFrame en formato .csv, en una ubicacion especificada.
  
  Argumentos:
  
    df= DataFrame pandas que se desea guardar.
    save_path= ruta donde se desea guardar el archivo.csv, en formato de texto.
    
  Excepciones:
  
    TypeError= Si el argumento df no es un DataFrame de pandas.
  """
  
  # Verificar que df sea un DataFrame.
  if not isinstance(df, pd.DataFrame):
    raise TypeError('el argumento "df" debe ser un DataFrame de pandas. ')

  # Crear el directorio si no existe
  directory = os.path.dirname(save_path)
  if directory and not os.path.exists(directory):
    os.makedirs(directory)

  try:
    #Guardar el DataFrame en formato csv
    df.to_csv(save_path, index=False)
    print('DataFrame guardado exitosamente en formato CSV.')
  except Exception as e:
    print(f'Error al guardar el DataFrame en formato CSV: {str(e)}')
